In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

import catboost as cb


In [4]:
data = pd.read_csv("train.csv")
data=data.dropna()

y=data.iloc[:,18]
X=data.drop("Promoted_or_Not",axis=1)
X=X.drop("EmployeeNo",axis=1)
X=X.drop("State_Of_Origin",axis=1)


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
X

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,No,Married,No,No,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,Yes,Married,No,No,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,Yes,Married,No,No,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,Yes,Single,No,No,1
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,Yes,Married,No,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38306,People/HR Management,First Degree or HND,Male,Agency and others,3,1987,12.5,2016,0,0,44,Yes,Married,No,No,1
38307,Information Technology and Solution Support,First Degree or HND,Female,Direct Internal process,2,1990,0.0,2018,0,0,70,Yes,Married,No,No,0
38308,Customer Support and Field Operations,"MSc, MBA and PhD",Female,Agency and others,2,1984,5.0,2013,0,0,48,Yes,Married,No,No,1
38309,Information and Strategy,First Degree or HND,Male,Agency and others,2,1994,12.5,2016,1,0,71,No,Married,No,No,3


In [6]:
ce_te = ce.TargetEncoder(cols=X.columns)
ce_te.fit(X_train,y_train)
X_train=ce_te.transform(X_train)
X_test=ce_te.transform(X_test)

/Users/hussami/opt/anaconda3/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
ce_baseN4 = ce.BaseNEncoder(cols=X.columns,base=16)
ce_baseN4.fit(X_train,y_train)

X_train=ce_baseN4.transform(X_train)
X_test=ce_baseN4.transform(X_test)

In [7]:
xgbr = xgb.XGBClassifier(max_depth=8,learning_rate=0.05,n_estimators=100,n_jobs=100,scale_pos_weight=0.9)
xgbr.fit(X_train,y_train)
y_predict = xgbr.predict(X_test)
score=f1_score(y_test,y_predict, average="micro")
print(score)

/Users/hussami/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:27:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9343523952504436


In [13]:
br = cb.CatBoostClassifier(learning_rate=0.3,n_estimators=100,max_depth=5,scale_pos_weight=0.5)


br.fit(X_train,y_train,verbose=0)


y_predict = br.predict(X_test)
score=f1_score(y_test,y_predict, average="micro")
print(score)

0.9325781356626177


In [14]:
rf = RandomForestClassifier(criterion='entropy',n_estimators=150,max_depth=7,n_jobs=-1)

rf.fit(X_train,y_train)  #fit on the data

rf_predict = rf.predict(X_test)

f1_score(y_test, rf_predict, average="micro")
print(score)

0.9325781356626177
